# Dynamically `operate` trades
### We will start with a simple cancellation of orders of the symbol, if there is an event fill
* [ ] Listen to orderfill event
* [ ] Cancel all contracts belonging to the orderfill
* [ ] Store the cancelled symbols
* [ ] Run nakeds on the cancelled symbols with new price on PAPER
* [ ] Make `renew_order` for the cancelled symbols

In [1]:
MARKET = 'SNP'

In [2]:
import sys
import pathlib
import numpy as np
import pandas as pd
import yaml
import asyncio

from ib_insync import IB, util, Option, MarketOrder, Contract
from typing import Callable, Coroutine, Union

In [3]:
# Specific to Jupyter. Will be ignored in IDE / command-lines
import IPython as ipy
if ipy.get_ipython().__class__.__name__ == 'ZMQInteractiveShell':
    import nest_asyncio
    nest_asyncio.apply()
    util.startLoop()
    pd.options.display.max_columns = None
    pd.options.display.float_format = '{:,.2f}'.format # set float precision with comma
    
    THIS_FOLDER = '' # Dummy for jupyter notebook's current folder
    BAR_FORMAT = "{l_bar}{bar:-20}{r_bar}"

In [4]:
# Get capability to import programs from `asyncib` folder
cwd = pathlib.Path.cwd() # working directory from where python was initiated
DATAPATH = cwd.joinpath('data', MARKET.lower()) # path to store data files
LOGFILE = cwd.joinpath(THIS_FOLDER, 'data', 'log', 'temp.log') # path to store log files

IBPATH = cwd.parent.parent.joinpath('asyncib') # where ib programs are stored

# append IBPATH to import programs.
if str(IBPATH) not in sys.path:  # Convert it to string!
    sys.path.append(str(IBPATH))
    
IBDATAPATH = IBPATH.joinpath('data', MARKET.lower())

In [5]:
# Get the host, port, cid
from engine import Vars

ibp = Vars(MARKET.upper())  # IB Parameters from var.yml
HOST, PORT, CID = ibp.HOST, ibp.PORT, ibp.MASTERCID

In [6]:
# Get the pickle files
from os import listdir
fs = listdir(DATAPATH)

files = [f for f in fs if f[-4:] == '.pkl']
for f in files:
    exec(f"{f.split('.')[0]} = pd.read_pickle(DATAPATH.joinpath(f))")
np.sort(np.array(files))

array(['df_chains.pkl', 'df_covers.pkl', 'df_nakeds.pkl', 'df_ohlcs.pkl',
       'df_opt_prices.pkl', 'df_opts.pkl', 'df_symlots.pkl',
       'df_unds.pkl', 'dfrq.pkl', 'qopt_rejects.pkl', 'qopts.pkl'],
      dtype='<U17')

In [7]:
# * IMPORTS
from ib_insync import IB, Order, LimitOrder
from support import quick_pf, get_openorders, get_prec, place_orders

In [8]:
# * FUNCTION INPUTS
MARKET = MARKET

In [9]:
# * SETUP
TMPLTPATH = pathlib.Path.cwd().joinpath(THIS_FOLDER, "data", "template")

In [10]:
# * SET THE CONNECTION
def connect(HOST, PORT, CID):
    try:
        if ib.isConnected():
            print(f"IB is already connected to host: {ib.client.host}, port:{ib.client.port}, clientId: {ib.client.clientId}")
        else:
            ib = IB().connect(HOST, PORT, CID)
            print(f"IB is set to host: {ib.client.host}, port:{ib.client.port}, clientId: {ib.client.clientId}")

    except NameError:
        try:
            ib = IB().connect(HOST, PORT, CID)
            print(f"IB connection status is now: {ib.isConnected()} at {ib.client.host}, port:{ib.client.port}, clientId: {ib.client.clientId}")
        
        except Exception:
            print(f"IB connection failed for {HOST}, port:{PORT}, clientId: {CID}")
            ib = None
    
    return ib

In [11]:
ib = connect(HOST, PORT, CID)

IB connection status is now: True at 127.0.0.1, port:1300, clientId: 10


In [12]:
# * REPRICE ORDERS UPON FILLS

def onExecDetails(trade, fill, SCALE=0.25):
    
    print(f"\nonExecDetails fired because of trade: {trade}\n\nfill: {fill}\n")
    
    # get the symbol of the trade filled
    try:
        symbol = {t.contract.symbol for t in trade}
    except TypeError:
        symbol = {trade.contract.symbol}
    
    # Download open orders and open trades
    ib.reqOpenOrders() 
    trades = ib.trades()
    
    # * TARGET DATAFRAME
    
    # . make the df
    df = util.df(t.contract for t in trades).iloc[:, :6]\
                .assign(contract=[t.contract for t in trades],
                        order=[t.order for t in trades],
                        status=[t.orderStatus.status for t in trades],
                        )\
                .join(util.df(t.order 
                              for t in trades).iloc[:, 2:7])\
                .rename(columns={'lastTradeDateOrContractMonth': 'expiry'})
    
    # . filter the df
    ACTIVE_STATUS = ['ApiPending', 'PendingSubmit', 'PreSubmitted', 'Submitted']
    mask = df.status.isin(ACTIVE_STATUS) & (df.action == "SELL") & (df.symbol.isin(symbol))
    df = df[mask]
    
    # . set the new price
    df['newLmt'] = np.where(df.action == 'SELL', 
                            df.lmtPrice + df.lmtPrice*(1+SCALE), 
                            df.lmtPrice - df.lmtPrice*(1-SCALE))
    
    df['newLmt'] = df['newLmt'].apply(lambda x: get_prec(x, ibp.PREC))
    
    # * CANCEL AND RE-ORDER
    
    # . cancel the orders first. These gives `Error validating request for VOL` (ref: )
    cancels = [ib.cancelOrder(o) for o in df.order]
    
    # . change order price to new limit price
    df = df.assign(order=[LimitOrder(action=action, totalQuantity=totalQuantity, lmtPrice=newLmt) 
                     for action, totalQuantity, newLmt 
                         in zip(df.action, df.totalQuantity, df.newLmt)])

    # . build the contract, orders and re-order
    cos = tuple(zip(df.contract, df.order))

    modified_trades = place_orders(ib = ib, cos = cos)
    
    return modified_trades

In [13]:
# set the event for order fills
ib.execDetailsEvent += onExecDetails

In [ ]:
# symbol = {t.contract.symbol for t in ib.fills()}
symbol = {'VIAC'}
SCALE = 0.25

In [ ]:
symbol